In [ ]:
import json

annotations_path = '/home/roberto/PythonProjects/S2RAWVessel/mmdetection/data/S2ESA/annotations/'
train_path = annotations_path + 'train.json'
val_path = annotations_path + 'val.json'
test_path = annotations_path + 'test.json'

# read json file
with open(train_path) as f:
    train = json.load(f)
with open(val_path) as f:
    val = json.load(f)
with open(test_path) as f:
    test = json.load(f)


In [ ]:
# gather statistics from this:
heights = [x['height'] for x in train['images']]
widths = [x['width'] for x in train['images']]
# bboxes
bboxes = [x['bbox'] for x in train['annotations']]



In [ ]:
print('Num of images train: ', len(train['images']))
print('Num of images val: ', len(val['images']))
print('Num of images test: ', len(test['images']))

print('Num of annotations train: ', len(train['annotations']))
print('Num of annotations val: ', len(val['annotations']))
print('Num of annotations test: ', len(test['annotations']))

In [ ]:
print('Mean width: ', sum(widths)/len(widths))
print('Mean height: ', sum(heights)/len(heights))

In [ ]:
print(max(heights))
print(max(widths))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def create_heatmap(bboxes, heatmap_size=(1000, 1000)):
    """
    Create a heatmap from a list of bounding boxes.

    Args:
        bboxes (list): List of bounding boxes. Each bounding box is a list of four floats [x, y, width, height].
        heatmap_size (tuple): Size of the heatmap. Default is (1000, 1000).

    Returns:
        None
    """
    heatmap = np.zeros(heatmap_size)

    for bbox in bboxes:
        x, y, width, height = map(int, bbox)
        heatmap[y:y+height, x:x+width] += 1

    plt.figure(figsize=(10,10), dpi=300)
    plt.imshow(heatmap, cmap='hot', interpolation='nearest')
    plt.axis(False)
    # add colorbar
    plt.colorbar()
    
    plt.show()


create_heatmap(bboxes, heatmap_size=(max(heights), max(widths)))


In [ ]:
# print the average dimension of the bbox, it scales with the image size
print('Mean width: ', sum([x[2] for x in bboxes])/len(bboxes))
print('Mean height: ', sum([x[3] for x in bboxes])/len(bboxes))

# Plot checkpoint:

In [ ]:
from pathlib import Path
import pandas as pd  
import os
import argparse
import matplotlib.pyplot as plt
from pathlib import Path
import subprocess
import shutil

In [ ]:
def plot_checkpoints(workdir):
    """
    This function plots the 'coco/bbox_mAP_50' values in a given directory. It reads a JSON file named 'scalars.json',
    and plots the 'coco/bbox_mAP_50' values against the 'step' values.

    Parameters:
    workdir (str): The directory path which contains the checkpoint files and 'scalars.json'.

    Returns:
    None: The function plots the graph and doesn't return any value.
    """
    scalars = list(Path(workdir).glob('**/scalars.json'))
    title = list(Path(workdir).glob('**/*.py'))
    title = title[0].stem
    data = pd.read_json(scalars[0], lines=True)
    data = data[data['coco/bbox_mAP_50'].notna()]

    # plot all the coco/bbox_mAP values and save each one in the same folder
    plt.figure(figsize=(5, 5))
    # add style to the plot:
    plt.style.use('seaborn-darkgrid')
    plt.xlabel('step')
    plt.ylabel('mAP@50')
    plt.title(title)
    plt.plot(data['step'], data['coco/bbox_mAP_50'])
    plt.show()

    # make a plot of the mAP values:
    plt.figure(figsize=(5, 5))
    # add style to the plot:
    plt.style.use('seaborn-darkgrid')
    plt.xlabel('step')
    plt.ylabel('mAP')
    plt.title(title)
    plt.plot(data['step'], data['coco/bbox_mAP'])
    plt.show()

    # make a plot of the mAP@75 values:
    plt.figure(figsize=(5, 5))
    # add style to the plot:
    plt.style.use('seaborn-darkgrid')
    plt.xlabel('step')
    plt.ylabel('mAP@75')
    plt.title(title)
    plt.plot(data['step'], data['coco/bbox_mAP_75'])
    plt.show()  
    
    
    # make a plot of mAP@50, mAP and mAP@75 in the same plot:
    plt.figure(figsize=(5, 5))
    # add style to the plot:
    plt.style.use('seaborn-darkgrid')
    plt.xlabel('step')
    plt.ylabel('mAP')
    plt.title(title)
    plt.plot(data['step'], data['coco/bbox_mAP_50'], label='mAP@50')
    plt.plot(data['step'], data['coco/bbox_mAP'], label='mAP')
    plt.plot(data['step'], data['coco/bbox_mAP_75'], label='mAP@75')
    plt.legend()
    plt.show()

In [ ]:
plot_checkpoints('/home/roberto/PythonProjects/S2RAWVessel/checkpoints/vfnet_r18_fpn_1x_vessel/20230518_152156_0.0005')

# Test Dataloader:

In [ ]:
from mmcv.transforms import LoadImageFromFile
import matplotlib.pyplot as plt
import cv2

# backend_args = dict(channel_order='bgr')
backend_args = None

loader = LoadImageFromFile(to_float32=True, color_type='color', imdecode_backend='tifffile', backend_args=backend_args)
loader

In [ ]:
from pathlib import Path  

immagini_path = Path('/home/roberto/PythonProjects/S2RAWVessel/mmdetection/data/S2ESA/imgs')
paths = list(immagini_path.glob('*.tiff'))

medie1, medie2, medie3 = [], [], []
dev1, dev2, dev3 = [], [], []
for idx,path in enumerate(paths):
    try:
        l = loader(results={'img_path': path})
        img = l['img']
        a,b,c = img[:,:,0], img[:,:,1], img[:,:,2]


        m1, m2, m3 = a.mean(), b.mean(), c.mean()
        s1, s2, s3 = a.std(), b.std(), c.std()
        
        medie1.append(m1)
        medie2.append(m2)
        medie3.append(m3)
        
        dev1.append(s1)
        dev2.append(s2)
        dev3.append(s3)
    except:
        print('Error with the png:', path)

In [26]:
import numpy as np 

print(np.mean(medie1))
print(np.mean(medie2))
print(np.mean(medie3))

print(np.mean(dev1))    
print(np.mean(dev2))
print(np.mean(dev3))



13275.393
13802.904
12052.766
10931.518
8811.33
7766.496


In [ ]:
import numpy as np

def mean_without_outliers(data):
    Q1 = np.percentile(data, 20)
    Q3 = np.percentile(data, 80)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    filtered_data = [x for x in data if lower_bound <= x <= upper_bound]

    return np.mean(filtered_data)

In [ ]:
print(mean_without_outliers(medie1))
print(mean_without_outliers(medie2))
print(mean_without_outliers(medie3))
print(mean_without_outliers(dev1))    
print(mean_without_outliers(dev2))
print(mean_without_outliers(dev3))

In [ ]:
# make an histogram of the mean values
plt.hist(medie1, bins=100)
plt.show()
# 

In [ ]:
plt.figure(figsize=(15, 15))
plt.imshow(img[:,:,:] / img.max())

In [25]:
r,g,b = img[:,:,0], img[:,:,1], img[:,:,2]

In [ ]:
from pathlib import Path
import pandas as pd  
import os

def manage_checkpoints(workdir):
    pesi = Path(workdir).glob('*.pth')
    scalars = list(Path(workdir).glob('**/scalars.json'))

    data = pd.read_json(scalars[0], lines=True)
    try:
        row = data[data['coco/bbox_mAP_50'] == data['coco/bbox_mAP_50'].max()]
        step, value = row['step'].values[0], row['coco/bbox_mAP_50'].values[0]
        print("step value")
        print(step, value)

        print(list(pesi))

        for p in pesi:
            nome = p.name
            print(nome)
            if nome != f'epoch_{step}.pth':
                # delete the file:
                print(f"Deleting {p}")
                os.remove(p)
    except KeyError:
        print("KeyError")
        for p in pesi:
            os.remove(p)
        
# Call the function with the directory path



In [ ]:
folders=["/home/roberto/PythonProjects/S2RAWVessel/checkpoints/20230516_055811_0.0005"]

for folder in folders:
    manage_checkpoints(folder)

In [ ]:
# make a plot of the mAP@50 values:
data = data[data['coco/bbox_mAP_50'].notna()]
plt.figure(figsize=(5, 5))
# add style to the plot:
plt.style.use('seaborn-darkgrid')
plt.xlabel('step')
plt.ylabel('mAP@50')
plt.plot(data['step'], data['coco/bbox_mAP_50'])
plt.show()

# make a plot of the mAP values:
plt.figure(figsize=(5, 5))
# add style to the plot:
plt.style.use('seaborn-darkgrid')
plt.xlabel('step')
plt.ylabel('mAP')
plt.plot(data['step'], data['coco/bbox_mAP'])
plt.show()

# make a plot of the mAP@75 values:
plt.figure(figsize=(5, 5))
# add style to the plot:
plt.style.use('seaborn-darkgrid')
plt.xlabel('step')
plt.ylabel('mAP@75')
plt.plot(data['step'], data['coco/bbox_mAP_75'])
plt.show()  

In [ ]:
data

# filter data when coco/bbox_mAP_50 is none:
data = data[data['coco/bbox_mAP_50'].notna()]
data

In [ ]:
a = ["/home/roberto/PythonProjects/S2RAWVessel/checkpoints/20230514_094247","/home/roberto/PythonProjects/S2RAWVessel/checkpoints/20230514_095528","/home/roberto/PythonProjects/S2RAWVessel/checkpoints/20230514_095528","/home/roberto/PythonProjects/S2RAWVessel/checkpoints/20230514_100835", "/home/roberto/PythonProjects/S2RAWVessel/checkpoints/20230514_104839"]

In [ ]:
def plot_checkpoints(workdir):
    """
    This function plots the 'coco/bbox_mAP_50' values in a given directory. It reads a JSON file named 'scalars.json',
    and plots the 'coco/bbox_mAP_50' values against the 'step' values.

    Parameters:
    workdir (str): The directory path which contains the checkpoint files and 'scalars.json'.

    Returns:
    None: The function plots the graph and doesn't return any value.
    """
    scalars = list(Path(workdir).glob('**/scalars.json'))
    title = list(Path(workdir).glob('**/*.py'))
    title = title[0].stem
    data = pd.read_json(scalars[0], lines=True)
    # plot all the coco/bbox_mAP values and save each one in the same folder
    data = data[data['coco/bbox_mAP_50'].notna()]

    plt.figure(figsize=(5, 5))
    # add style to the plot:
    plt.style.use('seaborn-darkgrid')
    plt.xlabel('step')
    plt.ylabel('mAP@50')
    plt.title(title)
    plt.plot(data['step'], data['coco/bbox_mAP_50'])
    plt.savefig(f'{workdir}/mAP_50.png')
    plt.show()

    # make a plot of the mAP values:
    plt.figure(figsize=(5, 5))
    # add style to the plot:
    plt.style.use('seaborn-darkgrid')
    plt.xlabel('step')
    plt.ylabel('mAP')
    plt.title(title)
    plt.plot(data['step'], data['coco/bbox_mAP'])
    plt.savefig(f'{workdir}/mAP.png')
    plt.show()

    # make a plot of the mAP@75 values:
    plt.figure(figsize=(5, 5))
    # add style to the plot:
    plt.style.use('seaborn-darkgrid')
    plt.xlabel('step')
    plt.ylabel('mAP@75')
    plt.title(title)
    plt.plot(data['step'], data['coco/bbox_mAP_75'])
    plt.savefig(f'{workdir}/mAP_75.png')
    plt.show()  

In [ ]:
import os
for folder in a:
    data = plot_checkpoints(folder)